In [1]:
%autosave 300

Autosaving every 300 seconds


In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd

from jupyterthemes import jtplot
jtplot.style()

In [174]:
d0 = pd.read_csv('data/turnstile_20200208.txt')
d1 = pd.read_csv('data/turnstile_20200502.txt')

d0.columns = [i.strip() for i in d0.columns]
d1.columns = [i.strip() for i in d1.columns]

In [177]:
print(d0.shape)
d0.head(2)

(205920, 11)


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/01/2020,03:00:00,REGULAR,7364079,2496560
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/01/2020,07:00:00,REGULAR,7364091,2496574


In [178]:
d0['date_time'] = d0.DATE + ' ' + d0.TIME
d1['date_time'] = d1.DATE + ' ' + d1.TIME

d0.date_time = pd.to_datetime(d0.date_time, infer_datetime_format=True)
d1.date_time = pd.to_datetime(d1.date_time, infer_datetime_format=True)

d0 = d0[d0.date_time.dt.dayofweek.isin([0, 1, 2, 3, 6])]
d1 = d1[d1.date_time.dt.dayofweek.isin([0, 1, 2, 3, 6])]

In [179]:
d0 = d0.groupby(['C/A', 'UNIT', 'SCP', 'DATE', 'STATION', 'LINENAME', 'DIVISION', 'DESC']).max().reset_index()
d1 = d1.groupby(['C/A', 'UNIT', 'SCP', 'DATE', 'STATION', 'LINENAME', 'DIVISION', 'DESC']).max().reset_index()

In [185]:
dfs = []
for key, grp in d0.groupby(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DESC']):
    temp = grp.copy()
    temp['en'] = temp.ENTRIES.diff()
    temp['ex'] = temp.EXITS.diff()
    dfs.append(temp)
d0 = pd.concat(dfs, sort=False)

dfs = []
for key, grp in d1.groupby(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DESC']):
    temp = grp.copy()
    temp['en'] = temp.ENTRIES.diff()
    temp['ex'] = temp.EXITS.diff()
    dfs.append(temp)
d1 = pd.concat(dfs, sort=False)

d0.dropna(subset=['en'], inplace=True)
d1.dropna(subset=['en'], inplace=True)

d0.en = np.where(d0.en >= 0, d0.en, 0)
d1.en = np.where(d1.en >= 0, d1.en, 0)

d0.ex = np.where(d0.ex >= 0, d0.ex, 0)
d1.ex = np.where(d1.ex >= 0, d1.ex, 0)

In [186]:
d0 = d0.groupby(['DATE', 'STATION', 'LINENAME', 'DIVISION', 'DESC']).sum().reset_index()
d1 = d1.groupby(['DATE', 'STATION', 'LINENAME', 'DIVISION', 'DESC']).sum().reset_index()

d0 = d0.groupby(['STATION', 'LINENAME', 'DIVISION', 'DESC']).median().reset_index()
d1 = d1.groupby(['STATION', 'LINENAME', 'DIVISION', 'DESC']).median().reset_index()

In [187]:
d0 = d0[['STATION', 'LINENAME', 'DIVISION', 'en', 'ex']]
d1 = d1[['STATION', 'LINENAME', 'DIVISION', 'en', 'ex']]

In [189]:
d1.head(2)

,STATION,LINENAME,DIVISION,en,ex
0,1 AV,L,BMT,1540.0,2576.5
1,103 ST,1,IRT,1085.0,980.5


In [190]:
d0.rename(columns={'en': 'FEB_ENTRIES', 'ex': 'FEB_EXITS'}, inplace=True)
d1.rename(columns={'en': 'APR_ENTRIES', 'ex': 'APR_EXITS'}, inplace=True)

In [191]:
d0.head(2)

,STATION,LINENAME,DIVISION,FEB_ENTRIES,FEB_EXITS
0,1 AV,L,BMT,18139.0,20145.0
1,103 ST,1,IRT,11931.0,5978.0


In [192]:
d0.shape, d1.shape

((494, 5), (488, 5))

In [194]:
d = d0.merge(d1)

In [195]:
print(d.shape)
d.head(2)

(498, 7)


,STATION,LINENAME,DIVISION,FEB_ENTRIES,FEB_EXITS,APR_ENTRIES,APR_EXITS
0,1 AV,L,BMT,18139.0,20145.0,1540.0,2576.5
1,103 ST,1,IRT,11931.0,5978.0,1085.0,980.5


In [234]:
d['line'] = d.LINENAME.apply(lambda x: ''.join(sorted(x)))

In [325]:
fix = pd.read_excel('data/fix_names.xlsx')

In [331]:
temp = fix.copy()
temp.dropna(subset=['data'], inplace=True)
temp.drop_duplicates(subset=['data'], inplace=True)
temp.index = temp.data
d.STATION = d.STATION.map(temp.new)

In [330]:
stations = pd.read_csv('data/NYC_Transit_Subway_Entrance_And_Exit_Data.csv')

stations.fillna('', inplace=True)

cols = ['Route1', 'Route2', 'Route3', 'Route4', 'Route5', 'Route6', 'Route7', 'Route8', 'Route9', 'Route10', 'Route11']
stations[cols] = stations[cols].astype(str)

stations['line'] = stations[cols].sum(axis=1)
stations['line'] = stations.line.apply(lambda x: ''.join(sorted(x)))

stations.line = stations.line.str.replace('.', '')
stations.line = stations.line.str.replace('0', '')

stations = stations[['Division', 'Line', 'Station Name', 'Station Latitude', 'Station Longitude', 'line']]

stations['Station Name'] = stations['Station Name'].str.upper()

stations.columns = ['DIVISION', 'line_name', 'STATION', 'lat', 'lon', 'line']

stations.STATION = stations.STATION.str.replace('(\d+)(\w+) ', r'\1 ')

temp = fix.copy()
temp.dropna(subset=['station'], inplace=True)
temp.drop_duplicates(subset=['station'], inplace=True)
temp.index = temp.station
stations.STATION = stations.STATION.map(temp.new)

stations.drop_duplicates(inplace=True)

In [344]:
stations = pd.read_csv('data/stations.csv')

stations.fillna('', inplace=True)

temp = fix.copy()
temp.dropna(subset=['station'], inplace=True)
temp.drop_duplicates(subset=['station'], inplace=True)
temp.index = temp.station

stations['STATION'] = stations.STOP.str.upper().map(temp.new)

stations.head()

,stop_id,DIVISION,line_name,STOP,routes,lat,lon,STATION
0,R01,BMT,Astoria,Astoria - Ditmars Blvd,N W,40.775036,-73.912034,ASTORIA - DITMARS BLVD
1,R03,BMT,Astoria,Astoria Blvd,N W,40.770258,-73.917843,ASTORIA BLVD
2,R04,BMT,Astoria,30 Av,N W,40.766779,-73.921479,30 AV
3,R05,BMT,Astoria,Broadway,N W,40.761820,-73.925508,BROADWAY
4,R06,BMT,Astoria,36 Av,N W,40.756804,-73.929575,36 AV


In [346]:
d.head(2)

,STATION,LINENAME,DIVISION,FEB_ENTRIES,FEB_EXITS,APR_ENTRIES,APR_EXITS,line
0,1 AV,L,BMT,18139.0,20145.0,1540.0,2576.5,L
1,103 ST,1,IRT,11931.0,5978.0,1085.0,980.5,1


In [354]:
df = d.merge(stations, on=['STATION', 'DIVISION'])

print(df.shape)
df.head()

(554, 14)


,STATION,LINENAME,DIVISION,FEB_ENTRIES,FEB_EXITS,APR_ENTRIES,APR_EXITS,line,stop_id,line_name,STOP,routes,lat,lon
0,1 AV,L,BMT,18139.0,20145.0,1540.0,2576.5,L,L06,Canarsie,1 Av,L,40.730953,-73.981628
1,103 ST,1,IRT,11931.0,5978.0,1085.0,980.5,1,119,Broadway - 7Av,103 St,1,40.799446,-73.968379
2,103 ST,1,IRT,11931.0,5978.0,1085.0,980.5,1,624,Lexington Av,103 St,6,40.790600,-73.947478
3,103 ST,6,IRT,13694.5,12627.0,2109.5,2708.5,6,119,Broadway - 7Av,103 St,1,40.799446,-73.968379
4,103 ST,6,IRT,13694.5,12627.0,2109.5,2708.5,6,624,Lexington Av,103 St,6,40.790600,-73.947478


In [3]:
df = pd.read_excel('output.xlsx')

print(df.shape)
df.head(2)

(450, 11)


,station,division,feb_entries,feb_exits,apr_entries,apr_exits,lines,lat,lon,geometry,line_name
0,1 AV,BMT,18139.0,20145.0,1540.0,2576.5,L,40.730953,-73.981628,"(40.730953,-73.981628)",Canarsie
1,103 ST,IRT,11931.0,5978.0,1085.0,980.5,1,40.799446,-73.968379,"(40.799446,-73.968379)",Broadway - 7Av


In [4]:
from shapely.geometry import Point

In [5]:
df['geometry'] = df.apply(lambda x: Point(x.lon, x.lat), 1)

In [6]:
gdf = gpd.GeoDataFrame(df)

In [ ]:
gdf['change_entries'] = gdf.apr_entries / gdf.feb_entries * 100
gdf['change_exits'] = gdf.apr_exits / gdf.feb_exits * 100

In [378]:
gdf.to_file('subways.shp')

In [11]:
gdf.to_file('subways.geojson', driver='GeoJSON')